In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from argparse import Namespace
import numpy as np
import pandas as pd
from tqdm import tqdm
from momentfm import MOMENTPipeline
import time
from datetime import timedelta
import matplotlib as plt
import os

# Configuration
config = Namespace(
    batch_size=32,
    num_epochs=50,
    learning_rate=1e-4,
    seq_len=101,  # Your sequence length
    n_channels=1,
    num_class=5,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

/home/chanan/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chanan/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
torch.cuda.is_available()

True

In [3]:
# Load the model
model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-large", 
    model_kwargs={
        'task_name': 'classification',
        'n_channels': config.n_channels,
        'num_class': config.num_class
    }
)
model.init()
model = model.to(config.device)

config.json: 100%|██████████| 951/951 [00:00<00:00, 1.81MB/s]
pytorch_model.bin: 100%|██████████| 1.39G/1.39G [00:32<00:00, 42.4MB/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 19.61 GiB of which 5.44 MiB is free. Process 295107 has 18.09 GiB memory in use. Including non-PyTorch memory, this process has 1.48 GiB memory in use. Of the allocated memory 1.30 GiB is allocated by PyTorch, and 9.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    """Plot and save confusion matrix"""
    save_dir=config.checkpoint_dir
    classes = config.classes_names
    try:
        # Create directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            print(f"Created directory: {save_dir}")
        
        print(f"\nCreating confusion matrix...")
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.colorbar()
        
        # Add class labels
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45, ha='right')
        plt.yticks(tick_marks, classes)
        
        # Add text annotations
        thresh = cm.max() / 2.
        for i, j in np.ndindex(cm.shape):
            plt.text(j, i, format(cm[i, j], 'd'),
                    horizontalalignment="center",
                    color="white" if cm[i, j] > thresh else "black")
        
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        
        
        # Save path with full directory
        save_path = os.path.join(save_dir, f'{config.model_name}_confusion_matrix.png')
        print(f"Saving confusion matrix to: {save_path}")
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print("Successfully saved confusion matrix")
        
        # Display the plot
        plt.show()
        
        # Close
        plt.close()
        
        
    except Exception as e:
        print(f"Error in plotting confusion matrix: {str(e)}")
        import traceback
        traceback.print_exc()


In [ ]:
def plot_metrics(train_losses, val_losses, train_accs, val_accs):
    """Plot and save training metrics"""
    save_dir=config.os.path.dirname(os.path.abspath(__file__))
    try:
        # Create directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            print(f"Created directory: {save_dir}")
        else:
            print(f"Directory already exists: {save_dir}")
        
        print("\nPlotting Training Metrics...")
        plt.figure(figsize=(12, 5))
        
        # Plot losses
        plt.subplot(1, 2, 1)
        plt.plot(train_losses, label='Training Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        
        # Plot accuracies
        plt.subplot(1, 2, 2)
        plt.plot(train_accs, label='Training Accuracy')
        plt.plot(val_accs, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy (%)')
        plt.legend()
        
        plt.tight_layout()
        
        
        # Save path with full directory
        save_path = os.path.join(save_dir, f'{config.model_name}_training_metrics.png')
        print(f"Saving metrics plot to: {save_path}")
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print("Successfully saved metrics plot")
        
        
        # Display the plot
        plt.show()
        
        #close
        plt.close()
    
    except Exception as e:
        print(f"Error in plotting metrics: {str(e)}")
        import traceback
        traceback.print_exc()

In [ ]:
def evaluate_model(model, val_loader):
    """
    Evaluate the model on test data
    
    Args:
        model: Trained model
        test_loader: DataLoader for test data
        config: Configuration object
        classes: List of class names (optional)
    """
    
    # Start timing
    start_time = time.time()
    
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            if config.use_gpu:
                batch_x = batch_x.float().cuda()
                batch_y = batch_y.long().cuda()
            
            # Create marking tensor with correct dimensions (batch_size, seq_len, 1)
            # batch_mark = torch.ones((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2]), device=batch_x.device)

            outputs = model(
                x_enc=batch_x,
                x_mark_enc= None,#batch_mark,
                x_dec=None,
                x_mark_dec=None
            )
            
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
            
            # all_preds.extend(predicted.cpu().numpy())
            # all_labels.extend(batch_y.cpu().numpy())
            
            all_preds.extend(predicted.cuda().numpy())
            all_labels.extend(batch_y.cuda().numpy())
    
    
    accuracy = 100 * correct / total
    print(f'Val Accuracy: {accuracy:.2f}%')
    
    end_time = time.time()
    eval_time = end_time - start_time
    
    # Plot confusion matrix
    plot_confusion_matrix(all_labels, all_preds, config)
    
    print(f"\nEvaluation Complete!")
    print(f"Total evaluation time: {timedelta(seconds=eval_time)}")
    
    return accuracy

In [ ]:
# Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc='Training')
    for batch_data, batch_labels in progress_bar:
        # # Reshape data to [batch_size, n_channels, seq_len]
        # batch_data = batch_data.unsqueeze(1)  # Add channel dimension
        
        # Move to device
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        # Forward pass
        outputs = model.classify(x_enc=batch_data)
        loss = criterion(outputs.logits, batch_labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        predictions = torch.argmax(outputs.logits, dim=1)
        correct += (predictions == batch_labels).sum().item()
        total += batch_labels.size(0)
        
        # Update total loss
        total_loss += loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{total_loss/(progress_bar.n+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return total_loss / len(train_loader), correct / total

# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc='Validation')
        for batch_data, batch_labels in progress_bar:
            # # Reshape data
            # batch_data = batch_data.unsqueeze(1)
            
            # Move to device
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device)
            
            # Forward pass
            outputs = model.classify(x_enc=batch_data)
            loss = criterion(outputs.logits, batch_labels)
            
            # Calculate accuracy
            predictions = torch.argmax(outputs.logits, dim=1)
            correct += (predictions == batch_labels).sum().item()
            total += batch_labels.size(0)
            
            total_loss += loss.item()
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f'{total_loss/(progress_bar.n+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    return total_loss / len(val_loader), correct / total

# Main training loop
def train_model(model, train_loader, val_loader, config):
    # Start timing
    total_start_time = time.time()

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(config.num_epochs):        
        print(f'\nEpoch {epoch+1}/{config.num_epochs}')

        # Epoch train time
        epoch_start_time = time.time()
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config.device
        )
        
        # Validate
        val_loss, val_acc = validate(
            model, val_loader, criterion, config.device
        )
        
        # Calculate epoch time
        epoch_time = time.time() - epoch_start_time
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%')
        print(f'Epoch time: {timedelta(seconds=int(epoch_time))}')

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model.state_dict()
            print(f'New best validation accuracy: {best_val_acc*100:.2f}%')

    # Calculate total training time
    total_time = time.time() - total_start_time
    print("\nTraining Complete!")
    print(f"Total training time: {timedelta(seconds=total_time)}")

     # Plot and save metrics
    plot_metrics(train_loss, val_loss, train_acc, val_acc)

    # Infrence
    evaluate_model(model, val_loader)
    

    return best_model

In [7]:
def drop_unused_features(df, num_features, pos):
    """
    Drops all columns except those at positions pos and then repeats for every num_features columns
    """
    # Get total number of columns
    total_cols = df.shape[1]
    # Calculate how many groups of 9 features we have
    num_groups = total_cols // num_features
    
    # Create a list of column indices to keep
    cols_to_keep = []
    for i in range(num_groups):
        inx = i * num_features + pos
        cols_to_keep.append(inx)
    
    # Keep only the selected columns
    df = df.iloc[:, cols_to_keep]
    reshaped_df = df.values.reshape(df.shape[0], df.shape[1], 1)
    
    return reshaped_df
feature_path = r'/home/chanan/Dataset/50ms/5/5 classes/features.csv'
label_path = r'/home/chanan/Dataset/50ms/5/5 classes/labels.csv'
feature_df = pd.read_csv(filepath_or_buffer=feature_path)
label_df = pd.read_csv(filepath_or_buffer=label_path)

reshaped_labels = label_df.values.reshape(-1)
print(reshaped_labels.shape)

# Clean the labels
def prepare_label(labels):
#     reshaped_labels = labels.values.reshape(-1)
    # Extract just the first number from each label string
    cleaned = np.array([int(label.split()[1]) for label in labels])
    y = cleaned.astype(np.int64)
    if len(y.shape) > 1:
        y = y.ravel()  # Flatten if needed
    return y

pos = 1
num_features = 9 
X = drop_unused_features(feature_df, num_features, pos)

y = prepare_label(reshaped_labels)  # Convert to int64 for PyTorch's CrossEntropyLoss

print(X.shape)
print(y.shape)
# print(y.shape)
# print(y)
# print(X)

(5737,)
(5737, 101, 1)
(5737,)


In [8]:
def prepare_data(X, y, config, val_split=0.2):
    batch_size = config.batch_size

    print("Original X shape:", X.shape)
    
    X = X.reshape(X.shape[0], 1, X.shape[1])
    print("Reshaped X shape:", X.shape)
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=val_split, random_state=32, stratify=y
    )
    
    # Create datasets - no need to reshape here since we did it above
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    
    # Adjust DataLoader settings based on device
    loader_args = {
        'batch_size': batch_size,
        'num_workers': 4 if torch.cuda.is_available() else 2,
        'pin_memory': torch.cuda.is_available(),
    }
    
    train_loader = DataLoader(train_dataset, shuffle=True, **loader_args)
    val_loader = DataLoader(val_dataset, shuffle=False, **loader_args)
    
    return train_loader, val_loader

In [9]:
# Prepare data
train_loader, val_loader = prepare_data(X, y, config)

Original X shape: (5737, 101, 1)
Reshaped X shape: (5737, 1, 101)


In [10]:
# Train model
best_model = train_model(model, train_loader, val_loader, config)

# Save the trained model
torch.save(best_model, 'best_moment_model.pth')


Epoch 1/50


Training:   0%|          | 0/144 [00:00<?, ?it/s]/home/chanan/miniconda3/envs/myenv/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/chanan/miniconda3/envs/myenv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Validation: 100%|██████████| 36/36 [00:01<00:00, 24.88it/s, loss=1.4848, acc=39.37%]


Train Loss: 1.5237, Train Acc: 38.48%
Val Loss: 1.4436, Val Acc: 39.37%
New best validation accuracy: 39.37%

Epoch 2/50


Validation: 100%|██████████| 36/36 [00:01<00:00, 24.12it/s, loss=1.3909, acc=41.38%]


Train Loss: 1.4026, Train Acc: 38.48%
Val Loss: 1.3522, Val Acc: 41.38%
New best validation accuracy: 41.38%

Epoch 3/50


...

Train Loss: 0.5858, Train Acc: 84.79%
Val Loss: 0.5334, Val Acc: 86.24%

Epoch 49/50


Validation: 100%|██████████| 36/36 [00:01<00:00, 23.30it/s, loss=0.5434, acc=86.50%]


Train Loss: 0.5792, Train Acc: 84.20%
Val Loss: 0.5283, Val Acc: 86.50%
New best validation accuracy: 86.50%

Epoch 50/50


Validation: 100%|██████████| 36/36 [00:01<00:00, 23.61it/s, loss=0.5382, acc=86.50%]


Train Loss: 0.5720, Train Acc: 85.01%
Val Loss: 0.5232, Val Acc: 86.50%
